In this task, we are to simulate a battle featured in the game "Risk", in which the attacker rolls three dice, and the defender rolls two. The highest two rolls from each side are then compared. If the attacker’s roll beats the defender’s, the defender loses a unit and if not the attacker does.

Lets use [np.random.randint](https://medium.com/@ai.mlresearcher/numpy-random-numbers-75b42907e9c4) to simulate rolling dice in just one battle.

In [35]:
import numpy as np
import matplotlib.pyplot as plt

# Idea from https://medium.com/@ai.mlresearcher/numpy-random-numbers-75b42907e9c4
# The attacker rolls three times and the defender twice
attackerturn = np.sort(np.random.randint(1, 7, size=3))
defenderturn = np.sort(np.random.randint(1, 7, size=2))

# We print the results of each roll
print("Attacker's turn:", attackerturn)
print("Defender's turn:", defenderturn)

Attacker's turn: [5 5 6]
Defender's turn: [5 5]
